In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import keras
from sklearn import metrics

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

# Convolutional Neural Network using Tensorflow

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Rescaling
from keras.utils import np_utils

#one hot encode
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# datagen = ImageDataGenerator(
#     featurewise_center = True,
#     samplewise_center = False,
#     featurewise_std_normalization=True,
#     samplewise_std_normalization=False,
#     zca_whitening=True,
#     rotation_range=90,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True,
#     rescale=None
# )

# datagen.fit(x_train)

Rescale = Rescaling(1./127.5, offset=-1)
x_test_r = Rescale(x_test)

In [ ]:
x_train_norm = x_train/255 - 0.5
x_test_norm = x_test/255 - 0.5

In [ ]:
from tensorflow.keras import datasets, layers, models


CNNmodel = models.Sequential()
#first param is number of filters
CNNmodel.add(layers.Conv2D(32, (3,3), input_shape=(32,32,3), padding='same'))
#CNNmodel.add(layers.LeakyReLU(alpha=0.35))
CNNmodel.add(layers.Activation("elu"))
CNNmodel.add(layers.BatchNormalization())


CNNmodel.add(layers.Conv2D(32, (3,3), padding='same'))
CNNmodel.add(layers.Activation("elu"))

#CNNmodel.add(layers.LeakyReLU(alpha=0.35))
CNNmodel.add(layers.BatchNormalization())

CNNmodel.add(layers.MaxPooling2D(2,2))
# CNNmodel.add(layers.BatchNormalization())
#CNNmodel.add(layers.Activation("relu"))
CNNmodel.add(layers.Dropout(0.4))


CNNmodel.add(layers.Conv2D(64, (3,3), padding='same'))
#CNNmodel.add(layers.LeakyReLU(alpha=0.35))
CNNmodel.add(layers.Activation("elu"))
CNNmodel.add(layers.BatchNormalization())

CNNmodel.add(layers.Conv2D(64, (3,3), padding='same'))
CNNmodel.add(layers.Activation("elu"))
CNNmodel.add(layers.BatchNormalization())

CNNmodel.add(layers.MaxPooling2D(2,2))
# CNNmodel.add(layers.BatchNormalization())
#CNNmodel.add(layers.LeakyReLU(alpha=0.35))
CNNmodel.add(layers.Dropout(0.4))

CNNmodel.add(layers.Conv2D(128, (3,3), padding='same'))
#CNNmodel.add(layers.LeakyReLU(alpha=0.35))
CNNmodel.add(layers.Activation("elu"))
CNNmodel.add(layers.BatchNormalization())

CNNmodel.add(layers.Conv2D(128, (3,3), padding='same'))
CNNmodel.add(layers.Activation("elu"))
CNNmodel.add(layers.BatchNormalization())

CNNmodel.add(layers.MaxPooling2D(2,2))
CNNmodel.add(layers.Dropout(0.4))

CNNmodel.summary()

In [ ]:
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(10, activation="softmax", kernel_regularizer="l2", bias_regularizer='l2'))
#CNNmodel.add(layers.Activation("relu"))

CNNmodel.summary()

In [ ]:
#CNNmodel.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
CNNmodel.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

#history = CNNmodel.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=14, validation_data=(x_test_r, y_test))
history = CNNmodel.fit(x_train_norm, y_train, batch_size=64, epochs=125, validation_data=(x_test_norm, y_test))

In [ ]:
import matplotlib.pyplot as plt

#plot to show each epochs validation accuracy and train accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
CNNtest = CNNmodel.predict(x_test_norm)
CNNtest_labels = np.argmax(CNNtest, axis=1)

y_test.reshape(10000,)
accuracy = metrics.accuracy_score(CNNtest_labels, y_test)
print('{:.2f}'.format(accuracy*100) + '%')

In [ ]:
from joblib import dump, load

dump(CNNmodel, 'cifar10.joblib')